In [43]:
import geopandas as gpd
import pandas as pd
import math
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon
from geographiclib.geodesic import Geodesic
from shapely.ops import split
from shapely import ops
from pyproj import Proj, Transformer

In [44]:
buildings= gpd.read_file('../tests/data/buildings.geojson')
road= gpd.read_file('../tests/data/road.geojson')

In [45]:
# for d in buildings.columns:
#     print(d)
road['end'] = None
road['start'] = None
road['start_point_x'] = None
road['start_point_y'] = None
road['end_point_x'] = None
road['end_point_y'] = None
road['length'] = None
road['bearing'] = None
road['mother_road']='None'

buildings['road_direction'] = 'None'
buildings['distance2_intersecting_road'] = 0
buildings['road_length_d_code'] = ''

# print(buildings)
# print(road)

In [48]:
class bdcPoint:
    	
	def __init__(self):
		
		self.x = 0
		self.y = 0

# Constant integers for directions
RIGHT = 1
LEFT = -1
ZERO = 0

def directionOfPoint(A, B, P):
    """[Provides direction of the point in respect to line]

    Args:
        A ([type]): [Starting Point of Line]
        B ([type]): [Ending Point of Line]
        P ([type]): [Point of Interest]

    Returns:
        [Direction(int)]: [Left,Right and Zero : -1,1,0 ( Left direction, Right Direction , On Line)]
    """
    global RIGHT, LEFT, ZERO
	
	# Subtracting co-ordinates of
	# point A from B and P, to
	# make A as origin
 
    B.x -= A.x
    B.y -= A.y
    P.x -= A.x
    P.y -= A.y

    # Determining cross Product
    """"The Cross-Product has an interesting Property which will be used to determine direction of a point from a line segment. That is, the cross-product of two points is positive if and only if the angle of those point at origin (0, 0) is in counter-clockwise. And conversely the cross-product is negative if and only if the angle of those point at origin is in clockwise direction."""
    cross_product = B.x * P.y - B.y * P.x
    # Return RIGHT if cross product is positive
    if (cross_product > 0):
        return RIGHT
        
    # Return LEFT if cross product is negative
    if (cross_product < 0):
        return LEFT

    # Return ZERO if cross product is zero
    return ZERO

def addDirection_BuildingCode(direction,length,index,road_index):
    #right-even, left-odd number 
    length=int(length)
    # print(length)
    if length < 60:
        associated_road=road.iloc[road_index]

        if associated_road.mother_road == 'start':
            mother_road_code=associated_road.start
        else :
            mother_road_code=associated_road.end

        mother_road= road.loc[road['NEW'] == mother_road_code]

        mr_geom= mother_road.geometry
        d = {'col1': ['name1'], 'geometry': [Point(associated_road.start_point_x, associated_road.start_point_y)]}
        motherroad_start_point = {'col1': ['name1'], 'geometry': [Point(mother_road.start_point_x, mother_road.start_point_y)]}
        

        gdf = gpd.GeoDataFrame(d, crs=4326)
        mother_df = gpd.GeoDataFrame(motherroad_start_point, crs=4326)
        
        sp_point=gdf.iloc[0]
        mother_st_p=mother_df.iloc[0]
        for line in mr_geom:
            # print(line)
            for l in line:
                mother_line=l
                break

        result=split(mother_line,sp_point.geometry)
        for r  in result:
            splitted_line=r
            splitted_line.srid = 4326
            if splitted_line.contains(mother_st_p.geometry):
                break
            else:
                continue
        
            
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)
        buildings.at[index,'road_length_d_code']=str(int(geom_transformed.length))+"/"
    if (direction == 1):
        # print("Right Direction")
        buildings.at[index,'road_direction']="right"
        buildings.at[index,'road_length_d_code']+=str(int(round_up_to_even(length)))
        
    elif (direction == -1):
        # print("Left Direction")
        buildings.at[index,'road_direction']="left"
        buildings.at[index,'road_length_d_code']+=str(int(round_up_to_odd(length)))
    else:
        # print("On  Line")

        buildings.at[index,'road_direction']="on_line"



            
            
            
def getStartEndPoint(line,type):
    if len(line)==1 or type=="road" :
       
        df_line = line.geometry.boundary
        # print(len(df_line))
        # print(line_bound)
        # df_line=line_bound.explode(index_parts=True)
        start_point,end_point=df_line[0],df_line[1]

        return start_point,end_point
    else:
        raise ValueError ("Line contains more than one code")

def getLineLength(line):
    projected_line = line.to_crs(epsg=32644)
    length= projected_line.geometry.length
    float_length=length.tolist()
    return float_length

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def round_up_to_odd(f):
    return math.ceil(f) // 2 * 2 + 1

def addStartEndPoint(start_point,end_point,index):
    
    road.at[index,'start_point_x']=float(start_point.x)
    road.at[index,'start_point_y']=float(start_point.y)
    
    road.at[index,'end_point_x']=float(end_point.x)
    road.at[index,'end_point_y']=float(end_point.y)

def ReviseStartEndDateWithBearing(x1,y1,x2,y2,bearing,index):
    # default direction is west to east , south to north 
    # print("ma revise garna aako dubai vettera")
    if bearing is None :
        bearing=0
    bearing=int(bearing)
    def assign_start():
        print("start")
        
        road.at[index,'mother_road']='start'
        road.at[index,'start_point_x']=float(x1)
        road.at[index,'start_point_y']=float(y1)
        road.at[index,'end_point_x']=float(x2)
        road.at[index,'end_point_y']=float(y2)
    def assign_end():
    # east_west/west_east direction , assign to east
        print("end")
        road.at[index,'mother_road']='end'
        road.at[index,'start_point_x']=float(x2)
        road.at[index,'start_point_y']=float(y2)
        road.at[index,'end_point_x']=float(x1)
        road.at[index,'end_point_y']=float(y1)
    
    if ((bearing >= 0) and (bearing<=45)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
        
    if ((bearing > 45) and (bearing<135)) :
        
        # east_west/west_east direction , assign to east
        assign_end()
    
    if ((bearing >= 135) and (bearing<=225)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
    if ((bearing > 225) and (bearing<315)) :
            
        # east_west/west_east direction , assign to east
        assign_end()
    if ((bearing >= 315) and (bearing<=360)) :
    #    North_South/south_north Direction, assign to south
        
        assign_start()

        

    
    

        
    

def FindIntersectedFeatures(start_point,end_point,line_row,index_road):
    for index, row in road.iterrows():
        line=row.geometry        
        if str(row.NEW) != str(line_row.NEW):
            if line.distance(start_point) < 1e-8 :
                # print("start found "+ str(row.NEW))
                # print("mero start vetiyo guys ")
                road.at[index_road,'start']=str(row.NEW)
    
            if line.distance(end_point) < 1e-8:
                # print("end found "+ str(row.NEW))
                road.at[index_road,'end']=str(row.NEW)
                # print("mero end vetiyo guys ")


def scan_I_feature(start_point,end_point,line_row,index_road):
    
        if ((line_row.start!= None) and (line_row.end== None)) :
            print("has start no end")
            road.at[index_road,'mother_road']='start'
            road.at[index_road,'start_point_x']=float(start_point.x)
            road.at[index_road,'start_point_y']=float(start_point.y)
            road.at[index_road,'end_point_x']=float(end_point.x)
            road.at[index_road,'end_point_y']=float(end_point.y)
        if ((line_row.start== None) and (line_row.end != None)) :
            print("has end no start")
            
            road.at[index_road,'mother_road']='end'
            road.at[index_road,'start_point_x']=float(end_point.x)
            road.at[index_road,'start_point_y']=float(end_point.y)
            road.at[index_road,'end_point_x']=float(start_point.x)
            road.at[index_road,'end_point_y']=float(start_point.y)
    
        if ((line_row.start!= None) and (line_row.end != None)) :
            print("has both")
            print(line_row.bearing)
        #both start and end exist which means line is in between roads 
            road.at[index_road,'mother_road']='None'
            ReviseStartEndDateWithBearing(start_point.x,start_point.y,end_point.x,end_point.y,line_row.bearing,index_road)

                    
def CalculateBearing(start_point,end_point,index):
    brng = Geodesic.WGS84.Inverse(start_point.y, start_point.x, end_point.y, end_point.x)['azi1']
    road.at[index,'bearing']=brng

def CalculateTouchLine():
    for index,row in road.iterrows():
        # print(row)
        start_point,end_point=getStartEndPoint(row,"road")
        addStartEndPoint(start_point,end_point,index)
        CalculateBearing(start_point,end_point,index)
        FindIntersectedFeatures(start_point,end_point,row,index)
        scan_I_feature(start_point,end_point,row,index)
        
        # break
    print("Sucessfully added : startpoint, endpoint , Start touching Line , End Touching Line , bearing ")

def assign_building_code():
    road['start_point_x'].apply(lambda x: float(x))
    road['start_point_y'].apply(lambda x: float(x))
    road['end_point_x'].apply(lambda x: float(x))
    road['end_point_y'].apply(lambda x: float(x))
    
    
    
    for index, row in buildings.iterrows():
        road_code = row.NEW
        road_linked_wrt_point= road.loc[road['NEW'] == road_code]
        line = road_linked_wrt_point.geometry
        
        point = row.geometry
        degree_length=line.project(point)
        # 5m=0.00005 in degree 
        perp_coord=line.interpolate(degree_length) #end_always 
        for r in perp_coord:
            single_p=r
            break
        
        try:
            d = {'col1': ['name1'], 'geometry': [Point(single_p.x, single_p.y)]}
        except:
            print(single_p)
            raise("found error")
            break
        try:
            st_point = {'col1': ['name1'], 'geometry': [Point(road_linked_wrt_point.start_point_x, road_linked_wrt_point.start_point_y)]}

            gdf = gpd.GeoDataFrame(d, crs=4326)
            st_df = gpd.GeoDataFrame(d, crs=4326)
            
        
            sp_point=gdf.iloc[0]
            st_point=st_df.iloc[0]
            
            for line in line:
                # print(line)
                for l in line:
                    mother_line=l
                    break
            result=split(mother_line,sp_point.geometry.buffer(0.0000001))
            for r  in result:
                    splitted_line=r
                    splitted_line.srid = 4326
                    if splitted_line.contains(st_point.geometry.buffer(0.0000001)):
                        break
                    else:
                        continue
            
                
            my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
            geom_transformed = ops.transform(my_transformer.transform, splitted_line)

            clipped_length=geom_transformed.length
            substracted_point=splitted_line.interpolate(splitted_line.length-0.0000001) #end_always 
            
            try:
                length= getLineLength(road_linked_wrt_point)
                buildings.at[index,'distance2_intersecting_road']=float(clipped_length)      
                try:
                    road_index=road_linked_wrt_point.index[0]
                    road.at[road_index,'length']=length[0]       
                    start = bdcPoint()
                    end = bdcPoint()
                    point = bdcPoint()
                    poi = row.geometry
                    # print(type(road_linked_wrt_point.start_point_x))
                    # try:
                    #     start.x,start.y,end.x,end.y,point.x,point.y=float(road_linked_wrt_point.start_point_x),float(road_linked_wrt_point.start_point_y),float(road_linked_wrt_point.end_point_x),float(road_linked_wrt_point.end_point_y),poi.x,poi.y
                    # except:
                    try:
                        start.x,start.y,end.x,end.y,point.x,point.y=float(substracted_point.x),float(substracted_point.y),float(perp_coord.x),float(perp_coord.y),poi.x,poi.y
                            
                        direction=directionOfPoint(start,end,point)
                        # print(direction)
                        # break
                        addDirection_BuildingCode(direction,clipped_length,index,road_index)
                    except:
                        # break
                        continue
                except:
                    continue
            except:
                continue
        except:
            continue
        # break
        
    print("-------Sucess ------ added: road_length_d_code , length of road , direction of building ")

    
                   


In [49]:
CalculateTouchLine()

/tmp/ipykernel_15191/3765522537.py:118: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  start_point,end_point=df_line[0],df_line[1]


has both
106.47456537561013
end
has end no start
has end no start
has end no start
has both
88.87358219914432
end
has end no start
has both
112.97185562118233
end
has both
94.52544515594666
end
has end no start
has end no start
has both
128.11027753858224
end
has end no start
has both
57.455109181993734
end
has end no start
has end no start
has both
75.85878320587629
end
has both
129.88344976525752
end
has end no start
has end no start
has end no start
has both
104.8141182668041
end
has end no start
has both
89.63706840661878
end
has both
98.40637923708172
end
has both
105.58318674581118
end
has end no start
has end no start
has end no start
has end no start
has both
105.77982551924534
end
has both
92.10398741006284
end
has both
97.60795842569236
end
has both
108.88387245484721
end
has end no start
has end no start
has both
108.47160246199815
end
has both
98.7418424313257
end
has end no start
has both
131.45135874977606
end
has both
96.45700828826381
end
has both
105.01798059988471
end

In [ ]:
assign_building_code()     

In [ ]:
buildings.to_file("../app_outputs/output_building.geojson", driver="GeoJSON")

In [ ]:
road.to_file("../app_outputs/output_road_1.geojson", driver="GeoJSON")